# Chapter 28: MEV (Maximal Extractable Value)

---

Maximal Extractable Value (MEV) refers to the maximum value that can be extracted from block production by including, excluding, or reordering transactions within a block. Originally known as "miner extractable value," the term evolved as Ethereum transitioned to proof-of-stake, acknowledging that validators (and searchers) capture this value. MEV has profound implications for users, protocols, and the decentralization of blockchain networks. In this chapter, we'll demystify MEV, explore common extraction strategies, and discuss how developers can protect their users and protocols from its harmful effects.

---

## 28.1 Understanding MEV

### 28.1.1 What is MEV?

MEV is the profit that a block producer (miner or validator) can make by manipulating the order of transactions in a block. Because block producers decide which transactions to include and in what order, they have the power to front-run, back-run, or censor transactions to capture value.

**Origins:** MEV was first studied in the context of Ethereum mining, where miners could reorder transactions to exploit arbitrage opportunities or liquidations. Today, MEV is a multi-billion dollar phenomenon, with searchers (specialized actors) competing to extract value by submitting transactions that are then included by validators, often via relayers like Flashbots.

**Why MEV exists:**
- **Block space is scarce**: Not all transactions can fit in a block.
- **Order matters**: The sequence of transactions affects outcomes (e.g., prices on DEXes).
- **Information asymmetry**: Block producers see pending transactions (the mempool) before they are confirmed.

### 28.1.2 How MEV is Extracted

The process typically involves:
1. **Mempool monitoring**: Searchers watch the public mempool for profitable opportunities (e.g., large trades, liquidations).
2. **Transaction simulation**: They simulate how their transaction would affect the state.
3. **Bundling**: They create a bundle of transactions (e.g., front-run + back-run) and submit it to a relay like Flashbots, paying the validator a fee (or bribe) for inclusion.
4. **Inclusion**: The validator includes the bundle, executing the MEV strategy.

In proof-of-stake, validators can also propose blocks themselves, but they often rely on searchers to identify opportunities.

### 28.1.3 Impact on Users

MEV can be both positive and negative:

**Negative impacts:**
- **Worse prices**: Users may receive worse execution due to front-running or sandwich attacks.
- **Failed transactions**: Searchers may cause transactions to fail, wasting user gas.
- **Censorship**: Validators could censor transactions that interfere with their MEV opportunities.
- **Network congestion**: Bots compete, driving up gas prices for everyone.

**Positive aspects:**
- **Arbitrage** can reduce price discrepancies across DEXes, improving market efficiency.
- **Liquidations** are necessary for lending protocols to remain solvent; MEV searchers perform them efficiently.

The challenge is to mitigate harmful MEV while preserving beneficial forms.

---

## 28.2 MEV Strategies

Searchers employ various strategies to extract value. Understanding them is key to designing countermeasures.

### 28.2.1 Front-Running

Front-running occurs when a searcher observes a pending transaction (e.g., a large buy order on a DEX) and places their own transaction with a higher gas price to get executed first. They then sell after the user's transaction pushes the price up, profiting from the price movement.

```
Front-Running Example:

Time:
1. User submits tx: Buy 10 ETH on Uniswap (price $3000)
2. Searcher sees it, submits tx: Buy 10 ETH with higher gas
3. Searcher's tx executes first, price rises to $3010
4. User's tx executes at $3010 (worse price)
5. Searcher sells at $3010 (or later), profiting from the price increase.
```

**Code illustration (simplified):**
```solidity
// Searcher contract
contract FrontRunner {
    function frontRun(address dex, uint amount) external {
        // Buy before user
        (bool success, ) = dex.call(abi.encodeWithSignature("buy(uint256)", amount));
        // ... later sell after user's tx
    }
}
```

### 28.2.2 Back-Running

Back-running is the opposite: placing a transaction right after a target transaction to benefit from its after-effects. For example, after a large trade that moves the price, a searcher can arbitrage across pools.

**Example:** A large trade on Uniswap creates a price discrepancy with Sushiswap. A back-runner immediately buys on the cheaper pool and sells on the more expensive one.

### 28.2.3 Sandwich Attacks

A sandwich attack combines front-running and back-running. The attacker places a buy order before the user's transaction (front-run) and a sell order after (back-run), effectively trading against the user.

```
Sandwich Attack:

1. User wants to buy token X with ETH on Uniswap.
2. Attacker sees this and places:
   - Buy X before user (drives price up)
   - Sell X after user (at higher price)
3. User buys at inflated price; attacker profits from the difference.
```

**Impact:** User gets a worse price; attacker profits at their expense. Sandwich attacks are common on DEXes with large slippage tolerance.

### 28.2.4 Arbitrage

Arbitrage exploits price differences between exchanges. Searchers monitor multiple DEXes and execute trades that bring prices back in line.

**Example:** ETH is $3000 on Uniswap, $3010 on Sushiswap. A searcher buys on Uniswap and sells on Sushiswap, pocketing the difference (minus gas and fees).

Arbitrage is generally considered beneficial because it improves market efficiency, but it can be harmful if it front-runs user trades.

### 28.2.5 Liquidations

In lending protocols (Aave, Compound), positions that fall below the collateralization ratio are liquidated. Liquidators repay part of the debt and receive a bonus (discounted collateral). MEV searchers compete to be the first to submit liquidation transactions, often bidding up gas prices.

**Example:** A user's ETH-backed loan becomes undercollateralized as ETH price drops. Searchers monitor the blockchain and submit liquidation transactions as soon as the condition is met, earning the liquidation bonus.

Liquidations are necessary for protocol health, but the race to liquidate can lead to gas wars and centralization of liquidators using MEV techniques.

---

## 28.3 MEV Protection

Developers can implement strategies to protect users from harmful MEV.

### 28.3.1 Flashbots and MEV-Boost

**Flashbots** is a research and development organization that created a communication channel between searchers and miners/validators, allowing them to submit bundles directly without exposing transactions to the public mempool. This reduces the harmful effects of front-running and gas wars.

**How Flashbots works (pre-merge):**
1. Searchers create bundles and submit to Flashbots relay.
2. Miners receive bundles and can include them in blocks without revealing them publicly.
3. Bundles are executed atomically and privately, preventing front-running.

**MEV-Boost** is the post-merge equivalent for proof-of-stake. It allows validators to outsource block construction to relays, which aggregate bundles from searchers. Validators receive the block and a payment (bribe) for including it.

```
MEV-Boost Flow:

Searchers → Bundles → Relays → Block Builder → Relay → Validator (proposes block)
```

**Benefits:**
- Reduces public mempool exposure.
- Democratizes MEV (smaller searchers can participate).
- Validators earn additional revenue.

### 28.3.2 Private Transaction Pools

Several services offer private transaction submission, where transactions are sent directly to miners/validators without entering the public mempool.

- **Flashbots Protect**: Users can submit transactions via Flashbots RPC; they are not broadcast to the public mempool.
- **BloxRoute (Eden Network)** : Similar private pools.
- **Taichi Network**: Private transaction relay.

**Integration in DApps:**
```javascript
// Sending a transaction via Flashbots RPC
const provider = new ethers.providers.JsonRpcProvider('https://rpc.flashbots.net');
// Use same as normal, but transactions go to Flashbots first.
```

### 28.3.3 Commit-Reveal Schemes

To prevent front-running, applications can use a commit-reveal pattern. Users first submit a commitment (hash) of their intended action, and later reveal it. Since the commitment is hidden, searchers cannot front-run.

**Example: Commit-reveal for a sealed-bid auction**
```solidity
contract SealedBidAuction {
    mapping(address => bytes32) public commitments;
    mapping(address => uint) public bids;

    function commit(bytes32 commitment) external {
        commitments[msg.sender] = commitment;
    }

    function reveal(uint bid, uint nonce) external {
        bytes32 commitment = keccak256(abi.encodePacked(msg.sender, bid, nonce));
        require(commitments[msg.sender] == commitment, "Invalid reveal");
        bids[msg.sender] = bid;
        // process auction...
    }
}
```

**Trade-offs:**
- Requires two transactions (higher gas).
- UX is more complex.
- Still vulnerable to front-running during reveal if not protected.

### 28.3.4 Fair Ordering Solutions

Some protocols aim to enforce fair transaction ordering:

- **Threshold encryption**: Transactions are encrypted and only decrypted after ordering (e.g., Chainlink's Fair Sequencing Services).
- **Timed commitments**: Users commit to a time delay, allowing others to react.
- **Batch auctions**: Instead of continuous order books, use discrete batch auctions where all orders in a batch are executed at a clearing price (e.g., Gnosis Protocol, CowSwap).

**CowSwap (CoW Protocol)** uses batch auctions and "coincidence of wants" to directly match orders, minimizing MEV.

---

## 28.4 MEV as a Developer

### 28.4.1 Designing MEV-Resistant Protocols

As a developer, you can design your protocol to minimize harmful MEV exposure.

**Best practices:**

1. **Use Commit-Reveal** for sensitive operations (e.g., auctions).
2. **Implement Slippage Protection**: Allow users to set minimum output amount (DEXes like Uniswap already do this).
3. **Use Time-Weighted Average Prices (TWAP)** instead of spot prices for oracles (prevents manipulation).
4. **Batch Processing**: Process transactions in batches (e.g., periodic settlement) to reduce ordering advantage.
5. **Limit Order Books**: Allow users to place limit orders that are executed at specified prices, reducing front-running risk.
6. **Integrate with MEV Protection** (Flashbots RPC) for user transactions.

**Example: Slippage protection in a swap function**
```solidity
function swap(uint amountIn, uint minAmountOut) external {
    uint amountOut = getAmountOut(amountIn);
    require(amountOut >= minAmountOut, "Slippage too high");
    // execute swap
}
```

### 28.4.2 Fair Ordering Solutions

Some projects are building infrastructure for fair ordering:

- **Chainlink FSS**: Fair Sequencing Services, a decentralized oracle network for ordering transactions.
- **Skip (Cosmos)** : MEV mitigation for Cosmos chains.
- **SUAVE**: Flashbots' project for a decentralized block builder marketplace.

As a developer, you can integrate these when they mature, but for now, the focus is on application-level mitigations.

---

## Chapter Summary

```
┌─────────────────────────────────────────────────────────────────┐
│                    CHAPTER 28 SUMMARY                           │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  MEV is the value extracted by block producers through          │
│  transaction reordering. It can be harmful or beneficial.       │
│                                                                 │
│  Common MEV strategies:                                         │
│    • Front-running: placing a tx before a target tx            │
│    • Back-running: placing after                              │
│    • Sandwich attacks: front + back run                       │
│    • Arbitrage: exploiting price differences                  │
│    • Liquidations: seizing undercollateralized loans          │
│                                                                 │
│  Protection mechanisms:                                        │
│    • Flashbots / MEV-Boost: private communication             │
│    • Private transaction pools                                │
│    • Commit-reveal schemes                                    │
│    • Slippage protection, TWAP, batch auctions                │
│                                                                 │
│  As a developer, design protocols with MEV resistance in mind:│
│    • Use slippage limits                                      │
│    • Consider commit-reveal                                   │
│    • Explore fair ordering solutions                          │
│    • Integrate with private relays                            │
│                                                                 │
│  MEV is an evolving field. Staying informed is key.           │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

**Next Chapter Preview:** Chapter 29 – Cross-Chain Development. We'll explore blockchain interoperability, bridges, and cross-chain messaging protocols like LayerZero, Wormhole, and Chainlink CCIP.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../7. testing_security_and_best_practices/27. gas_optimization.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='29. cross_chain_development.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
